In [3]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Document: {documents_df.iloc[doc_id]["documents"]}')
    print ('\n')
    print ('Similar Documents:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue

sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
subdirs = os.listdir("task1")
taskdir= "task1/001"
candidates_list = os.listdir(taskdir + '/candidates')
doc_list=[]
with open("task1/001/base_case.txt",mode="r",encoding="utf-8") as file:
    context = file.read()
    for candidatesfile in candidates_list:
        with open(taskdir+ '/candidates/' + candidatesfile, encoding="utf-8") as file:
            doc = [' '.join([word for line in file.readlines() for word in line.split()[:-1]])]
            doc_list.extend(doc)
            print(candidatesfile)
query_enbeddings = [sbert_model.encode(context)]
print(query_enbeddings)
#print(doc_list)
document_embeddings = sbert_model.encode(doc_list)
print(document_embeddings)
pairwise_similarities=cosine_similarity(query_enbeddings, document_embeddings)
rankings = np.flip(pairwise_similarities.argsort(), axis=1)
#pairwise_differences=euclidean_distances(document_embeddings)
print(rankings)
#most_similar(0,pairwise_similarities,'Cosine Similarity')
#most_similar(0,pairwise_differences,'Euclidean Distance')


001.txt
002.txt
003.txt
004.txt
005.txt
006.txt
007.txt
008.txt
009.txt
010.txt
011.txt
012.txt
013.txt
014.txt
015.txt
016.txt
017.txt
018.txt
019.txt
020.txt
021.txt
022.txt
023.txt
024.txt
025.txt
026.txt
027.txt
028.txt
029.txt
030.txt
031.txt
032.txt
033.txt
034.txt
035.txt
036.txt
037.txt
038.txt
039.txt
040.txt
041.txt
042.txt
043.txt
044.txt
045.txt
046.txt
047.txt
048.txt
049.txt
050.txt
051.txt
052.txt
053.txt
054.txt
055.txt
056.txt
057.txt
058.txt
059.txt
060.txt
061.txt
062.txt
063.txt
064.txt
065.txt
066.txt
067.txt
068.txt
069.txt
070.txt
071.txt
072.txt
073.txt
074.txt
075.txt
076.txt
077.txt
078.txt
079.txt
080.txt
081.txt
082.txt
083.txt
084.txt
085.txt
086.txt
087.txt
088.txt
089.txt
090.txt
091.txt
092.txt
093.txt
094.txt
095.txt
096.txt
097.txt
098.txt
099.txt
100.txt
101.txt
102.txt
103.txt
104.txt
105.txt
106.txt
107.txt
108.txt
109.txt
110.txt
111.txt
112.txt
113.txt
114.txt
115.txt
116.txt
117.txt
118.txt
119.txt
120.txt
121.txt
122.txt
123.txt
124.txt
125.txt


In [50]:
import spacy
from sentence_transformers import SentenceTransformer
def text_to_sent_list(text, 
                      nlp = spacy.load("en_core_web_lg"), 
                      embedder = SentenceTransformer('distilbert-base-nli-mean-tokens'),
                      min_len=1):
  
    ''' Returns cleaned article sentences and BERT sentence embeddings'''
    
    #convert to list of sentences
    text = nlp(text)
    sents = list(text.sents)
    
    #remove short sentences by threshhold                                                                                                
    sents_clean = [sentence.text for sentence in sents if len(sentence)> min_len]

    #remove entries with empty list
    sents_clean = [sentence for sentence in sents_clean if len(sentence)!=0]
    print(sents_clean)
    #sents_clean = [sentence.replace("\n"," ") for sentence in sents_clean]
    #embed sentences (deafult uses BERT SentenceTransformer)
    sents_embedding= np.array(embedder.encode(sents_clean, convert_to_tensor=True))
    
    return sents_clean, sents_embedding

In [27]:
'''
建構train id 檔案
'''
import json
import random
from collections import defaultdict

json_array = json.load(open("task1_train_2020_labels.json"))
subdirs = os.listdir("task1_clean_train_2020")
answers={}
for item in json_array:
    answers[item]=([int(os.path.splitext(i)[0]) for i in json_array[item]])
bm25_rank_dict = defaultdict(list)
train_query_docid_file = open("train_id.txt",mode="w")
with open("bm25_rank.txt",mode="r") as file: 
    for line in file.readlines():
        line = line.rstrip().split("\t")
        bm25_rank_dict[line[0]].append(line[1])
for task_id in subdirs:
    ans_list = answers[task_id]
    bm25_rank_list = bm25_rank_dict[task_id]
    bm25_rank_list = [id1 for id1 in bm25_rank_list if id1 not in ans_list]
    for i in ans_list:
        train_neg = random.sample(bm25_rank_list, 10)
        for j in train_neg:
            context = str(task_id) + "\t" + str(i) + "\t" + "1\n"
            train_query_docid_file.write(context)
            context = str(task_id) + "\t" + str(j) + "\t" + "0\n"
            train_query_docid_file.write(context)
train_query_docid_file.close()


In [2]:
'''
建構dev id 檔案
'''
import json
import random
from collections import defaultdict

json_array = json.load(open("task1_test_2020_labels.json"))
subdirs = os.listdir("task1_clean_test_bert_2020")
answers={}
for item in json_array:
    answers[item]=([int(os.path.splitext(i)[0]) for i in json_array[item]])
bm25_rank_dict = defaultdict(list)
train_query_docid_file = open("dev_id.txt",mode="w")
with open("bm25_rank.txt",mode="r") as file: 
    for line in file.readlines():
        line = line.rstrip().split("\t")
        bm25_rank_dict[line[0]].append(line[1])
for task_id in subdirs:
    ans_list = answers[task_id]
    bm25_rank_list = bm25_rank_dict[task_id]
    bm25_rank_list = [id1 for id1 in bm25_rank_list if id1 not in ans_list]
    for i in ans_list:
        train_neg = random.sample(bm25_rank_list, 10)
        for j in train_neg:
            context = str(task_id) + "\t" + str(i) + "\t" + "1\n"
            train_query_docid_file.write(context)
            context = str(task_id) + "\t" + str(j) + "\t" + "0\n"
            train_query_docid_file.write(context)
train_query_docid_file.close()


In [9]:
'''
針對query做text summarization
'''
import spacy
import numpy as np
import json
import random
from summa import summarizer
from nltk import sent_tokenize, word_tokenize

subdirs = os.listdir("task1_clean_train_bert_2020")
root_path = "task1_clean_train_bert_2020/"

for task_id in subdirs:
    taskdir= root_path+task_id
    candidates_list = os.listdir(taskdir + '/candidates')
    with open(root_path+ task_id+ "/base_case.txt", mode="r", encoding="utf-8") as file: 
        with open(root_path+ task_id+ "/clean_base_case.txt", mode="w", encoding="utf-8") as n_file:
            query_text = summarizer.summarize(file.read(),split=False,words=500)
            n_file.write(query_text)


In [10]:
'''
longformer
'''
import torch
from longformer.longformer import Longformer, LongformerConfig
from longformer.sliding_chunks import pad_to_window_size
from transformers import RobertaTokenizer

config = LongformerConfig.from_pretrained('longformer-base-4096/') 
# choose the attention mode 'n2', 'tvm' or 'sliding_chunks'
# 'n2': for regular n2 attantion
# 'tvm': a custom CUDA kernel implementation of our sliding window attention
# 'sliding_chunks': a PyTorch implementation of our sliding window attention
config.attention_mode = 'sliding_chunks'

model = Longformer.from_pretrained('longformer-base-4096/', config=config)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.model_max_length = model.config.max_position_embeddings

SAMPLE_TEXT = ' '.join(['Hello world! '] * 1000)  # long input document

input_ids = torch.tensor(tokenizer.encode(SAMPLE_TEXT)).unsqueeze(0)  # batch of size 1

# TVM code doesn't work on CPU. Uncomment this if `config.attention_mode = 'tvm'`
# model = model.cuda(); input_ids = input_ids.cuda()

# Attention mask values -- 0: no attention, 1: local attention, 2: global attention
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device) # initialize to local attention
attention_mask[:, [1, 4, 21,]] =  2  # Set global attention based on the task. For example,
                                     # classification: the <s> token
                                     # QA: question tokens

# padding seqlen to the nearest multiple of 512. Needed for the 'sliding_chunks' attention
input_ids, attention_mask = pad_to_window_size(
        input_ids, attention_mask, config.attention_window[0], tokenizer.pad_token_id)

output = model(input_ids, attention_mask=attention_mask)[0]
print(output)

tensor([[[-0.0473, -0.0016,  0.0404,  ..., -0.0328, -0.0960, -0.0306],
         [-0.2415,  0.2863,  0.1896,  ...,  0.0980,  0.0838,  0.1481],
         [-0.0653,  0.0544,  0.1262,  ..., -0.3759, -0.1101,  0.3571],
         ...,
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745],
         [-0.0236,  0.0741, -0.0145,  ..., -0.0990, -0.0409, -0.0745]]],
       grad_fn=<NativeLayerNormBackward>)


In [28]:
"""
產生text檔案
移除一些標號類似[1]這種
"""
import os
import unicodedata
import string
import nltk
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from gensim.summarization.bm25 import get_bm25_weights
import pathlib

lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())

subdirs = os.listdir("task1_train_2020")
root_path = "task1_train_2020/"
new_root_path = "task1_clean_train_bert_2020/"
#all_letters = string.ascii_letters + " .,;'"

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        #and c in all_letters
    )

for task_id in subdirs:
    documents = []
    queries = []
    taskdir= root_path+task_id
    candidates_list = os.listdir(taskdir + '/candidates')
    docs_list=[]
    pathlib.Path(new_root_path+ task_id+ '/candidates/').mkdir(parents=True, exist_ok=True) 
    new_file = open(new_root_path +task_id+"/base_case.txt",mode="w", encoding="utf-8")
    with open(root_path+ task_id+ "/base_case.txt", encoding="utf-8") as file:
        for line in file.readlines():    
            line=line.rstrip()
            line=unicodeToAscii(line)
            line=re.sub(r'\[.*?\]|\(.*?\)','',line)
            line=re.sub(r'\)|\.\.+', '', line)
            #line = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", line)
            if len(line) > 4:
                new_file.write(line+"\n")
    new_file.close()

    for candidatesfile in candidates_list:
        with open(root_path+task_id+ '/candidates/' + candidatesfile, encoding="utf-8") as file:
            with open(new_root_path+ task_id+ '/candidates/' + candidatesfile,mode="w", encoding="utf-8") as new_file:
                for line in file.readlines():    
                    line=line.rstrip()
                    line=unicodeToAscii(line)
                    line=re.sub(r'\[.*?\]|\(.*?\)','',line)
                    line=re.sub(r'\)|\.\.+', '', line)
                    #line = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", line)
                    if len(line) > 4:
                        new_file.write(line+"\n")
        
